# Time generating predictions for first 20 observations of test set

To compare inference time to the other two models, time the predictions for the first 20 observations of the test set.

## NowcastPNN with DOW

In [ ]:
%load_ext autoreload
%autoreload 3
from data_functions import get_dataset
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler as SRS
from train_utils import SubsetSampler as SS
from sklearn.model_selection import train_test_split as TTS
import torch, random, os, numpy as np
torch.use_deterministic_algorithms(True) # reproducibility

STATE = "SP"
WEEKS = False
TRIANGLE = True
PAST_UNITS = 40
MAX_DELAY = 40
BATCH_SIZE = 64
RANDOM_SPLIT = True
SEED = 1234
DEVICE = "mps"
DOW = True

dataset_dow = get_dataset(weeks=WEEKS, triangle=TRIANGLE, past_units=PAST_UNITS, max_delay=MAX_DELAY, state=STATE, dow = DOW)
#n_obs_40pu = len(dataset) # 2922 total dates, -39-39 for past_units and max_delay ->2844
## Define train and test indices
if RANDOM_SPLIT:
    all_idcs = range(dataset_dow.__len__())
    train_idcs, test_idcs = TTS(all_idcs, test_size=0.25, shuffle=True, random_state=SEED)
    train_idcs, val_idcs = TTS(train_idcs, test_size=0.25, shuffle=True, random_state=SEED)
    #train_idcs, test_idcs = [*range(600), *range(950, dataset.__len__())], [*range(600, 950)]
    VAL_BATCH_SIZE, TEST_BATCH_SIZE = len(val_idcs), len(test_idcs)
else:
    if WEEKS:
        train_idcs, test_idcs = range(300), range(300, dataset_dow.__len__())
        TEST_BATCH_SIZE = dataset_dow.__len__() - 300
    else: 
        train_idcs, test_idcs = range(int(0.75*dataset_dow.__len__())), range(int(0.75*dataset_dow.__len__()), dataset_dow.__len__()) # 2844 total obs - 711 test, still 25% even without random split dataset.__len__(), 2353
        train_idcs, val_idcs = TTS(train_idcs, test_size=0.25, shuffle=True, random_state=SEED)
        VAL_BATCH_SIZE, TEST_BATCH_SIZE = len(val_idcs), len(test_idcs)
        
## Define generator so sampling during training is deterministic and reproducible
g = torch.Generator()
g.manual_seed(SEED)
train_sampler_dow, val_sampler_dow, test_sampler_dow = SRS(train_idcs, generator=g), SRS(val_idcs), SS(test_idcs)
train_loader_dow, val_loader_dow, test_loader_dow = DataLoader(dataset_dow, batch_size=BATCH_SIZE, sampler=train_sampler_dow), DataLoader(dataset_dow, batch_size=VAL_BATCH_SIZE, sampler=val_sampler_dow, shuffle=False), DataLoader(dataset_dow, batch_size=TEST_BATCH_SIZE, sampler=test_sampler_dow, shuffle=False)

## Function to reset the sampler so each training run uses same order of observations for reproducibility
## Possible to define s.t. returns train_loader, but bc in notebook, possible to define globally
def regen_data():
    g = torch.Generator()
    g.manual_seed(SEED)
    global train_loader_dow
    train_loader_dow = DataLoader(dataset_dow, batch_size=BATCH_SIZE, sampler=SRS(train_idcs, generator=g))

def regen_loader(seed, dataset, batch_size, idcs):
    g = torch.Generator()
    g.manual_seed(seed)
    gen_loader = DataLoader(dataset, batch_size=batch_size, sampler = SRS(idcs, generator = g))
    return gen_loader

def set_seeds(SEED):
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    os.environ["PYTHONHASHSEED"] = str(SEED)
    random.seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True

set_seeds(SEED)


/Users/silaskoemen/Library/CloudStorage/OneDrive-ImperialCollegeLondon/Imperial/NowcastPNN/src/data_functions.py:108: DtypeWarning: Columns (7,11,23,45,46,47,55,65,69,75,86,102) have mixed types. Specify dtype option on import or set low_memory=False.
  dengdf = pd.read_csv(path, index_col=0)#pd.read_csv(f"../data/derived/DENG{state}.csv", index_col=0)


2922 2013-01-01 00:00:00 2020-12-31 00:00:00


In [2]:
from train_utils import train, EarlyStopper
from NowcastPNN import NowcastPNNDOW
set_seeds(SEED) # reproducible training runs
regen_data()
nowcast_pnn_dow = NowcastPNNDOW(past_units=PAST_UNITS, max_delay=MAX_DELAY, dropout_probs=[0.3, 0.1] if RANDOM_SPLIT else [0.15, 0.1]) # 0.3, 0.1 best for random split, 0.15, 0.1 for recent
nowcast_pnn_dow.load_state_dict(torch.load(f"./weights/weights-{PAST_UNITS}-{MAX_DELAY}-{'week' if WEEKS else 'day'}-fut0{'-rec' if not RANDOM_SPLIT else ''}{'-dow' if DOW else ''}"))

<All keys matched successfully>

In [9]:
test_obs = test_loader_dow.dataset[0]
for o in test_obs:
    if isinstance(o, tuple):
        for ob in o:
            print(ob, ob.shape)
    else:
        print(o, o.shape)

tensor([[0.0041, 0.0196, 0.0200,  ..., 0.0000, 0.0000, 0.0000],
        [0.0065, 0.0227, 0.0193,  ..., 0.0003, 0.0000, 0.0000],
        [0.0100, 0.0272, 0.0096,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0592, 0.0926, 0.0320,  ..., 0.0000, 0.0000, 0.0000],
        [0.0503, 0.0424, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0368, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       device='mps:0') torch.Size([40, 40])
tensor(2, device='mps:0') torch.Size([])
tensor([1253.], device='mps:0') torch.Size([1])


In [31]:
# Iterate through first 20 obs of test set, create list of times to compare
from time import time

NUM_TIMING_OBS = 20
N_SAMPLES = 200
inference_times = []

for i in range(NUM_TIMING_OBS):
    mat, y = test_loader_dow.dataset.__getitem__(test_loader_dow.sampler.indices[i])
    mat, dow_val = mat
    dow_val = torch.unsqueeze(dow_val.to("cpu"), 0)
    mat, y = torch.unsqueeze(mat.to("cpu"), 0), np.expand_dims(y.to("cpu").numpy(), 0)
    nowcast_pnn_dow.eval() # sets batch norm to eval so a single entry can be passed without issues of calculating mean and std and overall means and stds used
    nowcast_pnn_dow.drop1.train() # keeps dropout layers active
    nowcast_pnn_dow.drop2.train()
    nowcast_pnn_dow = nowcast_pnn_dow.to("cpu")
    start = time()
    preds = np.zeros((y.shape[0], N_SAMPLES))
    for i in range(N_SAMPLES):
        #preds[:, i] = np.squeeze(nowcast_pnn_dow(mat).sample().numpy())
        preds[:, i] = nowcast_pnn_dow(mat, dow_val).sample().numpy()
    min_preds, max_preds = np.min(preds, axis=1), np.max(preds, axis=1)
    pred_median = np.quantile(preds, 0.5, axis=1)
    inference_times.append(time() - start)

In [32]:
np.mean(inference_times), np.std(inference_times)

(0.1818275570869446, 0.014438801530890822)

### Time prediction of entire test set

In [33]:
(mat, dow), y = next(iter(test_loader_dow))
mat, dow = mat.to("cpu"), dow.to("cpu")
y = y.to("cpu").numpy()
nowcast_pnn_dow.eval() # sets batch norm to eval so a single entry can be passed without issues of calculating mean and std and overall means and stds used
nowcast_pnn_dow.drop1.train() # keeps dropout layers active
nowcast_pnn_dow.drop2.train()
nowcast_pnn_dow = nowcast_pnn_dow.to("cpu")
start_time = time()
preds = np.zeros((y.shape[0], N_SAMPLES))
for i in range(N_SAMPLES):
    preds[:, i] = nowcast_pnn_dow(mat, dow_val).sample().numpy()
min_preds, max_preds = np.min(preds, axis=1), np.max(preds, axis=1)
pred_median = np.quantile(preds, 0.5, axis=1)
print(time() - start_time)

13.865705966949463


## NowcastPNN without DOW

In [16]:
%load_ext autoreload
%autoreload 3
from data_functions import get_dataset
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler as SRS
from train_utils import SubsetSampler as SS
from sklearn.model_selection import train_test_split as TTS
import torch, random, os, numpy as np
torch.use_deterministic_algorithms(True) # reproducibility

STATE = "SP"
WEEKS = False
TRIANGLE = True
PAST_UNITS = 40
MAX_DELAY = 40
BATCH_SIZE = 64
RANDOM_SPLIT = True
SEED = 1234
DEVICE = "mps"
DOW = False

dataset = get_dataset(weeks=WEEKS, triangle=TRIANGLE, past_units=PAST_UNITS, max_delay=MAX_DELAY, state=STATE, dow = DOW)
#n_obs_40pu = len(dataset) # 2922 total dates, -39-39 for past_units and max_delay ->2844
## Define train and test indices
if RANDOM_SPLIT:
    all_idcs = range(dataset.__len__())
    train_idcs, test_idcs = TTS(all_idcs, test_size=0.25, shuffle=True, random_state=SEED)
    train_idcs, val_idcs = TTS(train_idcs, test_size=0.25, shuffle=True, random_state=SEED)
    #train_idcs, test_idcs = [*range(600), *range(950, dataset.__len__())], [*range(600, 950)]
    VAL_BATCH_SIZE, TEST_BATCH_SIZE = len(val_idcs), len(test_idcs)
else:
    if WEEKS:
        train_idcs, test_idcs = range(300), range(300, dataset.__len__())
        TEST_BATCH_SIZE = dataset.__len__() - 300
    else: 
        train_idcs, test_idcs = range(int(0.75*dataset.__len__())), range(int(0.75*dataset.__len__()), dataset.__len__()) # 2844 total obs - 711 test, still 25% even without random split dataset.__len__(), 2353
        train_idcs, val_idcs = TTS(train_idcs, test_size=0.25, shuffle=True, random_state=SEED)
        VAL_BATCH_SIZE, TEST_BATCH_SIZE = len(val_idcs), len(test_idcs)
        
## Define generator so sampling during training is deterministic and reproducible
g = torch.Generator()
g.manual_seed(SEED)
train_sampler, val_sampler, test_sampler = SRS(train_idcs, generator=g), SRS(val_idcs), SS(test_idcs)
train_loader, val_loader, test_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_sampler), DataLoader(dataset, batch_size=VAL_BATCH_SIZE, sampler=val_sampler, shuffle=False), DataLoader(dataset, batch_size=TEST_BATCH_SIZE, sampler=test_sampler, shuffle=False)

## Function to reset the sampler so each training run uses same order of observations for reproducibility
## Possible to define s.t. returns train_loader, but bc in notebook, possible to define globally
def regen_data():
    g = torch.Generator()
    g.manual_seed(SEED)
    global train_loader
    train_loader = DataLoader(dataset, batch_size=BATCH_SIZE, sampler=SRS(train_idcs, generator=g))

def regen_loader(seed, dataset, batch_size, idcs):
    g = torch.Generator()
    g.manual_seed(seed)
    gen_loader = DataLoader(dataset, batch_size=batch_size, sampler = SRS(idcs, generator = g))
    return gen_loader

def set_seeds(SEED):
    torch.manual_seed(SEED)
    np.random.seed(SEED)
    os.environ["PYTHONHASHSEED"] = str(SEED)
    random.seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True

set_seeds(SEED)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/Users/silaskoemen/Library/CloudStorage/OneDrive-ImperialCollegeLondon/Imperial/NowcastPNN/src/data_functions.py:108: DtypeWarning: Columns (7,11,23,45,46,47,55,65,69,75,86,102) have mixed types. Specify dtype option on import or set low_memory=False.
  dengdf = pd.read_csv(path, index_col=0)#pd.read_csv(f"../data/derived/DENG{state}.csv", index_col=0)


2922 2013-01-01 00:00:00 2020-12-31 00:00:00


In [18]:
from train_utils import train, EarlyStopper
from NowcastPNN import NowcastPNN
set_seeds(SEED) # reproducible training runs
regen_data()
nowcast_pnn = NowcastPNN(past_units=PAST_UNITS, max_delay=MAX_DELAY, dropout_probs=[0.3, 0.1] if RANDOM_SPLIT else [0.15, 0.1]) # 0.3, 0.1 best for random split, 0.15, 0.1 for recent
nowcast_pnn.load_state_dict(torch.load(f"./weights/weights-{PAST_UNITS}-{MAX_DELAY}-{'week' if WEEKS else 'day'}-fut0{'-rec' if not RANDOM_SPLIT else ''}{'-dow' if DOW else ''}"))

<All keys matched successfully>

In [26]:
# Iterate through first 20 obs of test set, create list of times to compare
from time import time

NUM_TIMING_OBS = 20
N_SAMPLES = 200
inference_times = []

for i in range(NUM_TIMING_OBS):
    mat, y = test_loader.dataset.__getitem__(test_loader.sampler.indices[i])
    mat, y = torch.unsqueeze(mat.to("cpu"), 0), np.expand_dims(y.to("cpu").numpy(), 0)
    nowcast_pnn.eval() # sets batch norm to eval so a single entry can be passed without issues of calculating mean and std and overall means and stds used
    nowcast_pnn.drop1.train() # keeps dropout layers active
    nowcast_pnn.drop2.train()
    nowcast_pnn = nowcast_pnn.to("cpu")
    start = time()
    preds = np.zeros((y.shape[0], N_SAMPLES))
    for i in range(N_SAMPLES):
        #preds[:, i] = np.squeeze(nowcast_pnn(mat).sample().numpy())
        preds[:, i] = nowcast_pnn(mat).sample().numpy()
    min_preds, max_preds = np.min(preds, axis=1), np.max(preds, axis=1)
    pred_median = np.quantile(preds, 0.5, axis=1)
    inference_times.append(time() - start)

In [27]:
np.mean(inference_times), np.std(inference_times)

(0.1423903226852417, 0.009843222814249606)

## Time prediction of entire test set


In [28]:
mat, y = next(iter(test_loader))
mat = mat.to("cpu")
y = y.to("cpu").numpy()
nowcast_pnn.eval() # sets batch norm to eval so a single entry can be passed without issues of calculating mean and std and overall means and stds used
nowcast_pnn.drop1.train() # keeps dropout layers active
nowcast_pnn.drop2.train()
nowcast_pnn = nowcast_pnn.to("cpu")
start_time = time()
preds = np.zeros((y.shape[0], N_SAMPLES))
for i in range(N_SAMPLES):
    #preds[:, i] = np.squeeze(nowcast_pnn(mat).sample().numpy())
    preds[:, i] = nowcast_pnn(mat).sample().numpy()
min_preds, max_preds = np.min(preds, axis=1), np.max(preds, axis=1)
pred_median = np.quantile(preds, 0.5, axis=1)
print(time() - start_time)

12.819590330123901
